# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> implement any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1634583074872_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
#export
from pyspark.sql.functions import col
from pyspark.sql import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-fall2021"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [4]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'smhasawade6'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [5]:
#export
def long_trips(trips):
    # Returns a Dataframe with Schema the same as :trips:
    filter_trips = trips.filter(trips.trip_distance >= 2)
    return filter_trips

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [6]:
#export
def manhattan_trips(trips, lookup):
    lookup = lookup.filter(col("Borough") =="Manhattan")
    top_trips = lookup.join(trips, lookup.LocationID == trips.DOLocationID, 'inner')    
    top_trips = top_trips.withColumn("Passenger_count", top_trips["Passenger_count"].cast("int"))
    top_trips = top_trips.groupBy("DOLocationID").sum('Passenger_count').select(col("DOLocationID"),col("sum(Passenger_count)").alias("pcount")).orderBy(["pcount"], ascending=[0])
    return top_trips.limit(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [7]:
#export
def weighted_profit(trips, mtrips): 
    weighted = trips.join(mtrips,mtrips.DOLocationID ==trips.DOLocationID, 'left_outer')
    proportion = trips.join(mtrips,mtrips.DOLocationID==trips.DOLocationID, 'inner')
    proportion = proportion.groupBy("PULocationID").count().select(col("PULocationID"),col("count").alias("proportion"))
    weighted = weighted.groupBy("PULocationID").agg({'total_amount':'avg', '*':'count'}).select(col("PULocationID"),col("count(1)").alias("count"),col("avg(total_amount)").alias("total_amount"))
    weighted = weighted.withColumnRenamed('PULocationID', 'PULocationID2')
    final = weighted.join(proportion,weighted.PULocationID2==proportion.PULocationID, 'inner')
    final = final.withColumn("weighted_profit", (col("total_amount")*col("proportion")/col("count")))
    final = final.select(col("PULocationID"),col("weighted_profit").cast("decimal(30,15)"))
    return final

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [8]:
#export
def final_output(calc, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    final = calc.join(lookup, calc.PULocationID==lookup.LocationID,"inner")
    final = final.select("Zone", "Borough", "weighted_profit").orderBy(["weighted_profit"],ascending=[0])
    return final.limit(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
    Test your code on the small dataset first, as the large dataset will take a significantly longer time to run
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> use the same bucket url for multiple runs of the `main()` function, as this will cause errors. Make sure to change the name of your output location every time. (ie: s3://cse6242-gburdell3/output-large2)
</div>

Update the below cell with the path to your bucket, then run the below cell to run your code to store the results in S3.

When you have tested the code with the small dataset, run it again using the large dataset. Your large output file will appear in a folder in your s3 bucket called YOUROUTPUT.csv as a csv file with a name something like part-0000-4d992f7a-0ad3-48f8-8c72-0022984e4b50-c000.csv. Download this file and rename it to q3_output.csv for submission. Do not make any other changes to the file. 

In [9]:
# update your bucket path
bucket = 's3://cse6242-smhasawade6/output-large4'
main('large', bucket)

#bucket = 's3://cse6242-smhasawade6/output-small'
#main('small', bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

smhasawade6
Trip Count:  187203269
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens|29.360455779130856|
|Flushing Meadows-...|       Queens|27.304845733617675|
|       South Jamaica|       Queens|26.294916239873476|
|     Randalls Island|    Manhattan|24.150989940227518|
|        Astoria Park|       Queens|21.706417112147516|
|Briarwood/Jamaica...|       Queens|19.945064631789343|
|Springfield Garde...|       Queens|19.468309288781903|
|             Jamaica|       Queens|19.283943000137896|
|              Corona|       Queens|18.228769248155980|
|   LaGuardia Airport|       Queens|18.181338808373000|
|         Jamaica Bay|       Queens|17.100529446757893|
|             Maspeth|       Queens|17.005450640079545|
|Eltingville/Annad...|Staten Island|16.837764756944450|
|         JFK Airport|       Queens|16.777725348249643|
|        Batt

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading
</div>

In [26]:
print(user())
trips, lookup = load_data(size='small')
trips = long_trips(trips)
mtrips = manhattan_trips(trips, lookup)
wp = weighted_profit(trips, mtrips)
final = final_output(wp,lookup)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

smhasawade6
Trip Count:  7667792
+------------+------------------+
|PULocationID|   weighted_profit|
+------------+------------------+
|         148| 7.697406151367027|
|         243| 7.934042506248551|
|          65| 6.630698230351501|
|         133| 2.915089453360257|
|         126| 5.502195387030551|
|          81| 4.005536332179930|
|          76| 3.656237522182793|
|         192| 3.407708576894463|
|          12|10.554036603239672|
|         209| 9.291515922788959|
|         122| 7.362530578512398|
|         140| 8.288846291439944|
|         177| 2.199673930059335|
|           1| 4.242351020408163|
|         205| 6.969148919294074|
|         206|21.491893491124262|
|           6|31.784444444444436|
|         142| 7.905672364297694|
|           3| 5.461863425925925|
|         178| 4.461430078727376|
+------------+------------------+
only showing top 20 rows